# A Demo of General Parsers for QM jobs

Author: Xiaorui Dong

Currently, there are 3 built-in QM parsers for Gaussian, QChem, and ORCA, respectively. However, when encountering other QM software packages, e.g., Molpro, Psi4, Turbomole, a user may try the general parser `CclibLog` for help.

Notes:
- If you see a warning in an interactive cell indicating you need to install py3dmol while you have already do so, don't worry, the warning message should disappear once you drag the slider or change the selections.

In [ ]:
from rdmc.external.logparser import CclibLog
from rdtools.view import base_viewer, mol_viewer

%load_ext autoreload
%autoreload 2

## 1. Load the file

In [ ]:
############ EXAMPLES #################
# opt
log = 'data/psi4-opt.out'
# freq
# log = 'data/psi4-freq.out'
######################################
clog = CclibLog(log)
print('QM Software:', clog.cclib_results.metadata['package'].lower())
print(clog.job_type)

In [ ]:
## Optional
# you may also modify the `job_type` to enable analysis of scan or irc, if the job type is parsed wrongly
# clog._job_type = ['opt', 'freq', 'scan', 'irc']

## 2. Converged?

If the optimization was converged, then show the converged geometry


In [ ]:
try:
    print(f'Optimization done?: {clog.cclib_results.optdone}')
except AttributeError:
    print('This is not an OPT job')

xyz = clog.get_best_opt_geom(xyz_str=True)
base_viewer(xyz, 'xyz').update()

# XYZ format
print('The optimized XYZ:\n')
print(xyz)

## 3. Convergence analysis

Check the trend for each convergence criterion
- `logy`: plot the y axis in log scale
- `relative`: Plot the relative value to the convergence criteria

In [ ]:
clog.plot_opt_convergence(logy=True, relative=True)

### 3.1 Interact with opt job

In [ ]:
clog.interact_opt()

## 4. Frequencies

In [ ]:
assert hasattr(clog.cclib_results, 'vibfreqs'), "Not a frequency job"
print(f'Number of freqs: {clog.freqs.shape[0]}')
print(', '.join([str(f) + ' cm^-1' for f in clog.freqs]))
print(f'Number of negative freqs: {clog.num_neg_freqs}')
print(f'Negative freqs: {clog.neg_freqs}')

In [ ]:
clog.interact_freq()

# According to our test, the cclib has an issue parsing `freq-psi4.out`'s geometries:
# AttributeError: 'ccData_optdone_bool' object has no attribute 'atomcoords'
# However, this function should work probably for other files